<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/E-commerce%20search%20engine%20with%20Sentence%20Transformer%20SKELETON/E_commerce_search_engine_with_Sentence_Transformer_SKELETON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers datasets

In [4]:
!pip install aicrowd-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 7.0 MB/s eta 0:00:00
  Created wheel for pyzmq: filename=pyzmq-22.1.0-cp310-cp310-linux_x86_64.whl size=1365959 sha256=f40eed1b103ffe7afd84ac6353494d7964c5d148104a8d988bba3b216902a2d8
  Stored in directory: /root/.cache/pip/wheels/2d/e1/f6/45f

In [8]:
import pandas as pd
import csv
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
import numpy as np

# **DATASET DOWNLOAD**

In [6]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/4FqmKuqj9uLXixN4USgV7t1X1GVlBFkK4I6gKoHJg5s
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/4FqmKuqj9uLXixN4USgV7t1X1GVlBFkK4I6gKoHJg5s'
API Key valid
Gitlab access token valid
Saved details successfully!


In [7]:
!aicrowd dataset download -c esci-challenge-for-improving-product-search

product_catalogue-v0.3.csv.zip: 100% 328M/328M [00:20<00:00, 15.7MB/s]
sample_submission_public-v0.3.csv.zip: 100% 331k/331k [00:00<00:00, 584kB/s]
test_public-v0.3.csv.zip: 100% 394k/394k [00:00<00:00, 697kB/s]
train-v0.3.csv.zip: 100% 6.80M/6.80M [00:01<00:00, 5.95MB/s]
product_catalogue-v0.3.csv.zip: 100% 657M/657M [00:27<00:00, 24.2MB/s]
sample_submission_public-v0.3.csv.zip: 100% 812k/812k [00:00<00:00, 1.13MB/s]
test_public-v0.3.csv.zip: 100% 2.94M/2.94M [00:00<00:00, 2.98MB/s]
train-v0.3.csv.zip: 100% 19.8M/19.8M [00:01<00:00, 11.4MB/s]
product_catalogue-v0.3.csv.zip: 100% 657M/657M [00:54<00:00, 12.2MB/s]
sample_submission_public-v0.3.csv.zip: 100% 803k/803k [00:00<00:00, 946kB/s]
test_public-v0.3.csv.zip: 100% 2.94M/2.94M [00:02<00:00, 1.29MB/s]
train-v0.3.csv.zip: 100% 20.3M/20.3M [00:02<00:00, 9.44MB/s]


In [8]:
!unzip "/content/train-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/train-v0.3.csv.zip
  inflating: /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv  


In [9]:
!unzip "/content/product_catalogue-v0.3.csv.zip" -d "/content/dataset/"

Archive:  /content/product_catalogue-v0.3.csv.zip
  inflating: /content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv  


# **DATASET PREPARATION**

In [10]:
filepath_train = "/content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv"

In [11]:
filepath_catalogue = "/content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv"

In [12]:
df_catalogue = pd.read_csv(filepath_catalogue)

In [13]:
df_train = pd.read_csv(filepath_train)

In [14]:
df_train

,example_id,query,product_id,query_locale,substitute_label
0,0,11 degrees,B079VKKJN7,es,no_substitute
1,1,11 degrees,B079Y9VRKS,es,no_substitute
2,2,11 degrees,B07D2DDCZH,es,no_substitute
3,3,11 degrees,B07DP4LM9H,es,no_substitute
4,4,11 degrees,B07G37B9HP,es,no_substitute
...,...,...,...,...,...
1834739,1834739,針なしほっちきす,B08XGQ9RH7,jp,substitute
1834740,1834740,針なしほっちきす,B0987RGRF2,jp,no_substitute
1834741,1834741,針なしほっちきす,B099NFJWP6,jp,no_substitute
1834742,1834742,針なしほっちきす,B09F3B413J,jp,no_substitute


In [15]:
set(df_train['substitute_label'])

{'no_substitute', 'substitute'}

In [16]:
df_catalogue

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),NaN,NaN,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,NaN,NaN,11 Degrees,NaN,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,NaN,NaN,11 Degrees,Negro (,es
...,...,...,...,...,...,...,...
1815211,B09FJYBVKS,スポーツブラ 揺れない ヨガウェア レディース トップス 通気性 ナイトブラ スポーツブラジ...,<p>【注意事項】</p><p>※商品内容以外のアクセサリー・小物等は付属しません.</p>...,シンプルのデザインで合わせやすいです。コート、半袖、シャツを合わせて、家で着てもいい、寝てい...,Doworspaw,ベージュ,jp
1815212,B09FTJ3S1N,ブラジャー レディース 補正ブラ Gabrioir 揺れない 大きいサイズ バストアップ美胸...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp
1815213,B09G72LDQZ,レディース ブラジャー 補正ブラ トップス Gabrioir 大きいサイズ 揺れない 脇高設...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,ベージュ,jp
1815214,B09GBGRTPB,ナイトブラ ノンワイヤーブラ ブラジャー レディース Gabrioir 大きいサイズ パット...,<p>【配送】</p><p>普通には2-3日後出荷できますが。出荷した後15-25日ぐらい届...,【いろんな場合に適用】軽い運動時、家事、外出、長時間移動、温泉旅行などに活用なセクシルームブ...,Gabrioir,パープル,jp


In [17]:
df_train.loc[df_train['query_locale'] == 'us']

,example_id,query,product_id,query_locale,substitute_label
249721,249721,revent 80 cfm,B000MOO21W,us,no_substitute
249722,249722,revent 80 cfm,B001E6DMKY,us,no_substitute
249723,249723,revent 80 cfm,B003O0MNGC,us,no_substitute
249724,249724,revent 80 cfm,B00MARNO5Y,us,no_substitute
249725,249725,revent 80 cfm,B00XBZFWWM,us,no_substitute
...,...,...,...,...,...
1522342,1522342,zephyr polishing kit,B081SYK6R2,us,no_substitute
1522343,1522343,zephyr polishing kit,B087HZQY4V,us,no_substitute
1522344,1522344,zephyr polishing kit,B08H4ZJ6Q1,us,substitute
1522345,1522345,zephyr polishing kit,B08LSN8MT8,us,substitute


## **DATASET CREATION FROM THE 2 DATAFRAMES**

In [18]:
def get_product_title(product_id):
  options = []
  options.append(str(product_id))
  print(options)
  print("======================================")
  matched_row = df_catalogue.loc[df_catalogue['product_id'].isin(options)]
  print(matched_row)
  print("======================================")
  print(matched_row['product_title'])
  print("======================================")
  print(str(matched_row['product_title']).split("   "))
  print("======================================")
  print(int(str(matched_row['product_title']).split("   ")[0]))
  print("======================================")
  return matched_row['product_title'][int(str(matched_row['product_title']).split("   ")[0])]

In [19]:
get_product_title("B08LSN8MT8") #SUBSTITUTE 	zephyr polishing kit

['B08LSN8MT8']
        product_id                                      product_title  \
933994  B08LSN8MT8  Kshineni Car Foam Drill 3-Inch Buffing Pad,11 ...   

                                      product_description  \
933994  <b>🚗Kshineni Car Foam Drill 3Inch Buffing Pad,...   

                                     product_bullet_point product_brand  \
933994  🚗Package Includes: 8 pcs 3" Sponge Polishing P...      Kshineni   

       product_color_name product_locale  
933994                NaN             us  
933994    Kshineni Car Foam Drill 3-Inch Buffing Pad,11 ...
Name: product_title, dtype: object
['933994', ' Kshineni Car Foam Drill 3-Inch Buffing Pad,11 ...\nName: product_title, dtype: object']
933994


'Kshineni Car Foam Drill 3-Inch Buffing Pad,11 Pcs Polishing Pads Kit,Car Buffer Polisher Kit Drill Buffing Kit for Car Polishing,Waxing,Sealing Glaze'

In [20]:
get_product_title("B087HZQY4V") #NO SUBSTITUTE  	zephyr polishing kit

['B087HZQY4V']
        product_id                                      product_title  \
933992  B087HZQY4V  Zephyr Orange Ruffy Heavy Cut Clear Dip Airway...   

                               product_description  \
933992  Orange Ruffy Heavy Cut Clear Dip Airway 10   

                                product_bullet_point product_brand  \
933992  Orange Ruffy Heavy Cut Clear Dip Airway 10""        Zephyr   

       product_color_name product_locale  
933992                NaN             us  
933992    Zephyr Orange Ruffy Heavy Cut Clear Dip Airway...
Name: product_title, dtype: object
['933992', ' Zephyr Orange Ruffy Heavy Cut Clear Dip Airway...\nName: product_title, dtype: object']
933992


'Zephyr Orange Ruffy Heavy Cut Clear Dip Airway 10""'

In [21]:
def get_product_title(product_id):
  options = []
  options.append(str(product_id))
  matched_row = df_catalogue.loc[df_catalogue['product_id'].isin(options)]
  #print(int(str(matched_row['product_title']).split("   ")[0]))
  return matched_row['product_title'][int(str(matched_row['product_title']).split("   ")[0])]

In [22]:
header = ['query', 'product', 'label']

for j in range(10):
  file_path = "/content/csv_file_" + str(j) + ".csv"
  with open(file_path, 'w', encoding = 'UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(header)

    for i in range(j * 10000, (j * 10000) + 2000):
      data=[df_train[i : i + 1]['query'][i], get_product_title(df_train[i : i + 1]['product_id'][i]), df_train[i : i + 1]['substitute_label'][i]]
      writer.writerow(data)


In [11]:
filepaths = ["/content/csv_file_" + str(i) + ".csv" for i in range(10)]
print(filepaths)

['/content/csv_file_0.csv', '/content/csv_file_1.csv', '/content/csv_file_2.csv', '/content/csv_file_3.csv', '/content/csv_file_4.csv', '/content/csv_file_5.csv', '/content/csv_file_6.csv', '/content/csv_file_7.csv', '/content/csv_file_8.csv', '/content/csv_file_9.csv']


In [12]:
dataset = load_dataset('csv', data_files = filepaths)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'product', 'label'],
        num_rows: 20000
    })
})

In [26]:
dataset['train'][1]

{'query': '11 degrees',
 'product': 'Camiseta Eleven Degrees Core TS White (M)',
 'label': 'no_substitute'}

## **DATA PREPROCESSING**

In [14]:
def get_label(label):
  if label == 'substitute':
    return 1.0
  else:
    return 0.0

In [15]:
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
max_length = 64

In [17]:
def preprocess(dataset):
  if dataset['product'] == None:
    dataset['product'] = dataset['query']

  dataset['input_ids_query'] = []
  dataset['token_type_ids_query'] = []
  dataset['attention_mask_query'] = []

  dataset['input_ids_product'] = []
  dataset['token_type_ids_product'] = []
  dataset['attention_mask_product'] = []

  tokenized_output_query = tokenizer(dataset['query'], max_length = max_length, padding = 'max_length', truncation = True)
  tokenized_output_product = tokenizer(dataset['product'],max_length = max_length, padding = 'max_length', truncation = True)

  dataset['input_ids_query'].append(tokenized_output_query['input_ids'])
  dataset['token_type_ids_query'].append(tokenized_output_query['token_type_ids'])
  dataset['attention_mask_query'].append(tokenized_output_query['attention_mask'])

  dataset['input_ids_product'].append(tokenized_output_product['input_ids'])
  dataset['token_type_ids_product'].append(tokenized_output_product['token_type_ids'])
  dataset['attention_mask_product'].append(tokenized_output_product['attention_mask'])

  dataset['label'] = get_label(dataset['label'])

  return dataset

In [18]:
prep_dataset = dataset.map(preprocess)

In [32]:
prep_dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'product', 'label', 'input_ids_query', 'token_type_ids_query', 'attention_mask_query', 'input_ids_product', 'token_type_ids_product', 'attention_mask_product'],
        num_rows: 20000
    })
})

In [33]:
prep_dataset['train'][21]

{'query': '144 preservativos',
 'product': 'Durex Preservativos Dame Placer con Puntos y Estrías - Total 12 Condones',
 'label': 1.0,
 'input_ids_query': [[0,
   51332,
   122785,
   26465,
   2,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]],
 'token_type_ids_query': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]],
 'attention_mask_query': [[1,
   1,
  

In [19]:
batch_size = 128

In [20]:
tf_dataset = prep_dataset["train"].to_tf_dataset(
    columns = ['input_ids_query', 'token_type_ids_query', 'attention_mask_query','input_ids_product', 'token_type_ids_product', 'attention_mask_product', 'label'],
    shuffle = True,
    batch_size = batch_size,
)

In [36]:
tf_dataset

<_PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.float32, name=None), 'input_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_query': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'input_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'token_type_ids_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None), 'attention_mask_product': TensorSpec(shape=(None, 1, 64), dtype=tf.int64, name=None)}>

In [37]:
for i in tf_dataset.take(1):
  print(i)

{'label': <tf.Tensor: shape=(128,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)>, 'input_ids_query': <tf.Tensor: shape=(128, 1, 64), dtype=int64, numpy=
array([[[     0,  19178,  52224, ...,      1,      1,      1]],

       [[     0,      6,  13001, ...,      1,      1,      1]],

       [[     0,  19138,   4241, ...,      1,      1,      1]],

       ...,

       [[     0,  82936,      7, ...,      1,      1,      1]],

       [[   

# **MODELING**

In [21]:
model = TFAutoModel.from_pretrained(model_id)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  117653760 
                                                                 
Total params: 117653760 (448.81 MB)
Trainable params: 117653760 (448.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
class SentenceTransformer(Model):
  def __init__(self, model):
    super(SentenceTransformer, self).__init__()
    self.model = model
    self.dense = Dense(1, activation = "sigmoid")

  def compile(self, optimizer, loss_fn):
    super(SentenceTransformer, self).compile()
    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.loss_metric = tf.keras.metrics.Mean(name = "loss")

  @property
  def metrics(self):
    return [self.loss_metric]

  def mean_pooling(self, model_output, attention_mask): #We use attention mask to not take paddings into account to calculate mean

    token_embeddings = model_output[0] #0th element contains the embeddings

    attention_mask_expanded = tf.cast(
        tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
        #we use tf.broadcast_to to make tf.expand_dims(attention_mask, -1) the same shape as token_embeddings
        tf.float32
    )

    sum = tf.math.reduce_sum(token_embeddings * attention_mask_expanded, axis = 1)

    num_elements = tf.math.reduce_sum(attention_mask_expanded, axis = 1)

    mean = sum / tf.clip_by_value(num_elements, 1e-9, tf.float32.max)
    #we clip by value to avoid division by a value less than 1e-9 (like 0) or greater than maximum float32 number

    return mean

  def train_step(self, train_data):

    query = {'input_ids' : train_data['input_ids_query'][:, 0, :],
           'token_type_ids' : train_data['token_type_ids_query'][:, 0, :],
           'attention_mask' : train_data['attention_mask_query'][:, 0, :]}

    product = {'input_ids' : train_data['input_ids_product'][:, 0, :],
             'token_type_ids' : train_data['token_type_ids_product'][:, 0, :],
             'attention_mask' : train_data['attention_mask_product'][:, 0, :]}

    labels = train_data['label']

    with tf.GradientTape() as recorder:

      query_predictions = self.model(query)
      product_predictions = self.model(product)

      pooled_query_pred = self.mean_pooling(query_predictions, train_data['attention_mask_query'][:, 0, :])
      pooled_product_pred = self.mean_pooling(product_predictions, train_data['attention_mask_product'][:, 0, :])

      pred_concat = tf.concat([pooled_query_pred, pooled_product_pred, tf.abs(pooled_query_pred - pooled_product_pred)], axis = -1)

      model_predictions = self.dense(pred_concat)

      loss = self.loss_fn(labels, model_predictions)

    partial_derivatives = recorder.gradient(loss, self.model.trainable_weights)
    self.optimizer.apply_gradients(zip(partial_derivatives, self.model.trainable_weights))

    self.loss_metric.update_state(loss)

    return {'loss' : self.loss_metric.result()}

# **TRAINING**

In [40]:
sentence_transformer = SentenceTransformer(model)

In [41]:
sentence_transformer.model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  117653760 
                                                                 
Total params: 117653760 (448.81 MB)
Trainable params: 117653760 (448.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
sentence_transformer.compile(optimizer = tf.keras.optimizers.Adam(2e-5), loss_fn = tf.keras.losses.BinaryCrossentropy())

In [46]:
history = sentence_transformer.fit(tf_dataset, epochs = 6)

Epoch 1/6


157/157 [==============================] - 247s 999ms/step - loss: 0.4006
Epoch 2/6
157/157 [==============================] - 159s 1s/step - loss: 0.2869
Epoch 3/6
157/157 [==============================] - 159s 1s/step - loss: 0.2207
Epoch 4/6
157/157 [==============================] - 158s 1s/step - loss: 0.1499
Epoch 5/6
157/157 [==============================] - 158s 1s/step - loss: 0.0950
Epoch 6/6
157/157 [==============================] - 158s 1s/step - loss: 0.0703


In [47]:
model_path = '/content/sentence_transformers.h5'

In [48]:
sentence_transformer.model.save_weights(model_path)

# **TESTING**

## **CREATE EMBEDDINGS FOR PRODUCTS**

In [23]:
filepath_catalogue = "/content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv"
df_catalogue = pd.read_csv(filepath_catalogue)

In [24]:
product_titles = [str(df_catalogue['product_title'][i]) for i in range(len(df_catalogue))] #list of all product titles

In [51]:
product_titles

['11 Degrees de los Hombres Playera con Logo, Negro, L',
 'Camiseta Eleven Degrees Core TS White (M)',
 '11 Degrees de los Hombres Core Pull Over Hoodie, Azul, S',
 '11 Degrees Poli Panel Track Pant XL Black',
 '11 Degrees Gorra Trucker Negro OSFA (Talla única para Todos sexos)',
 '11 Degrees de los Hombres Optum Poly Joggers, Negro, XL',
 '11 Degrees Core Zip Poly Top S Black',
 '11 Degrees Camiseta De Núcleo M Hot Red',
 '11 Degrees Trucker Cap - Black & White',
 '11 Degrees Chaqueta Espacial S Black',
 '11 Degrees Chaqueta Espacial S Blk Wht',
 '11 Degrees de los Hombres Pantalones de chándal Ajustados con Texto de Odin, Negro, S',
 '11 Degrees de los Hombres Odin Text Hoodie, Negro, S',
 '11 Degrees 11º Odin Long Sleeve Logo Top Camiseta para Hombre - algodón Talla: M',
 '11 Degrees de los Hombres Joggers Pitillo de Doble Cintura con vellón, Negro, M',
 'Durex Preservativos Saboreame con Sabores Afrutados - Pack 144 Condones',
 'Preservativos Pasante sensibles, Pack de 144 (500666)

In [52]:
print(len(product_titles))

1815216


In [25]:
embeddings = []

In [26]:
inference_batch_size = 640

In [55]:
print(len(product_titles) // inference_batch_size)

2836


In [27]:
def mean_pooling(model_output, attention_mask): #We use attention mask to not take paddings into account to calculate mean

  token_embeddings = model_output[0] #0th element contains the embeddings

  attention_mask_expanded = tf.cast(
      tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
      #we use tf.broadcast_to to make tf.expand_dims(attention_mask, -1) the same shape as token_embeddings
      tf.float32
  )

  sum = tf.math.reduce_sum(token_embeddings * attention_mask_expanded, axis = 1)

  num_elements = tf.math.reduce_sum(attention_mask_expanded, axis = 1)

  mean = sum / tf.clip_by_value(num_elements, 1e-9, tf.float32.max)
  #we clip by value to avoid division by a value less than 1e-9 (like 0) or greater than maximum float32 number

  return mean

In [ ]:
for i in range(len(product_titles) // inference_batch_size):
  tokenized_output = tokenizer(
      product_titles[inference_batch_size * i : inference_batch_size * (i + 1)],
      max_length = max_length,
      padding = 'max_length',
      truncation = True,
      return_tensors = "tf")
  model_output = model(tokenized_output)
  embedding = mean_pooling(model_output = model_output, attention_mask = tokenized_output['attention_mask'])
  embeddings.append(embedding)

  if i % 100 == 0:
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


In [61]:
embeddings

[<tf.Tensor: shape=(640, 384), dtype=float32, numpy=
 array([[ 0.05826151,  0.14133458, -0.29005146, ..., -0.321197  ,
          0.09974635, -0.22156633],
        [ 0.03517564,  0.31928387,  0.05435671, ...,  0.01177065,
         -0.18750703, -0.50677633],
        [-0.12945235,  0.33699957, -0.1135186 , ..., -0.25594392,
          0.15845747,  0.04102493],
        ...,
        [-0.25843406,  0.39152262,  0.0887166 , ..., -0.49570206,
         -0.09055951,  0.16729397],
        [-0.02648067, -0.42649072, -0.20272745, ...,  0.3790894 ,
          0.01441738, -0.29877827],
        [-0.05780469,  0.04500363, -0.00495596, ..., -0.07639268,
          0.24523656, -0.3804063 ]], dtype=float32)>,
 <tf.Tensor: shape=(640, 384), dtype=float32, numpy=
 array([[-0.04484354, -0.1577699 , -0.28252226, ...,  0.07477917,
          0.3707531 ,  0.15627846],
        [-0.15015446,  0.08255205,  0.07602035, ...,  0.39741182,
          0.412279  , -0.05600003],
        [-0.18676747, -0.04253494, -0.3665152 ,

In [28]:
np.savez_compressed('embeddings.npz', embeddings)
np.savez_compressed('product_titles.npz', product_titles)

## **USE EMBEDDINGS TO MAKE INFERENCE FOR QUERY**

In [35]:
loaded_embedding = np.load('/content/embeddings.npz')
loaded_embedding_array = np.array(loaded_embedding['arr_0'])

In [4]:
import numpy as np

In [29]:
loaded_titles = np.load('/content/product_titles.npz')
loaded_titles_array = np.array(loaded_titles['arr_0'])

In [ ]:
loaded_embedding_array.shape

In [ ]:
loaded_embedding_array = loaded_embedding_array.reshape(-1,loaded_embedding_array.shape[2])
print(loaded_embedding_array.shape)

In [ ]:
inputs = tokenizer(["keyboard for laptop"], max_length = max_length, padding = 'max_length', truncation = True, return_tensors = "tf")
#inputs = tokenizer(["device for typing on laptop"], max_length = max_length, padding = 'max_length', truncation = True, return_tensors = "tf")
logits = model(**inputs)
out_embedding = mean_pooling(logits, inputs['attention_mask'])
print(out_embedding.shape)

In [ ]:
u_dot_v = np.matmul(loaded_embedding_array, (np.array(out_embedding).T))
print(u_dot_v.shape)

In [ ]:
u_magnitude = np.sqrt(np.sum(loaded_embedding_array * loaded_embedding_array, axis = -1))
print(u_magnitude.shape)
print(u_magnitude)

In [ ]:
v_magnitude = np.sqrt(np.sum(out_embedding * out_embedding, axis = -1))
print(v_magnitude.shape)
print(v_magnitude)

In [ ]:
cosine_similarity = u_dot_v.T / (u_magnitude * v_magnitude)
print(cosine_similarity)

In [ ]:
sorted_indices = np.argsort(cosine_similarity, axis = -1)
print(sorted_indices)

In [ ]:
for i in range(25): #Top 25 results (products)
  print(i, loaded_titles_array[sorted_indices[:, len(sorted_indices[0]) - i - 1]])